In [ ]:
# Считайте набор данных

df_fraud = pd.read_csv
df_empl = pd.read_csv('employee.csv')

## OneHot

In [ ]:
# Закодируйте категориальные признаки числами 0 и 1 с помощью OneHotEncoder
# Выделите отдельные датасеты с закодированными признаками
#   train -> fit_transform
#   test -> transform

empl_encoder = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')

X_empl_train_onehot = empl_encoder.fit_transform(X_empl_train[empl_cat_feat])
X_empl_test_onehot = empl_encoder.transform(X_empl_test[empl_cat_feat])

# StandardScaler

#### **Как использовать [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) с обучающей и тестовой выборками**

**Основные принципы:**

* Обучайте (fit) скейлер только на обучающих данных.

* Применяйте преобразование (transform) к обучающим и тестовым данным.

* Не обучайте scaler на полной выборке (до разделения), это приведет к ["утечке информации" (data leakage)](https://scikit-learn.org/stable/common_pitfalls.html#data-leakage).

* [Используйте одинаковое преобразование для train и test](https://scikit-learn.org/stable/common_pitfalls.html#inconsistent-preprocessing).

In [ ]:
# Масштабируйте количественные признаки
#   train -> fit_transform
#   test -> transform

diab_scaler = StandardScaler().set_output(transform='pandas')

num_cols = diab_num_feat  # список числовых столбцов, создан ранее

X_diab_train_scaled = X_diab_train.copy()
X_diab_test_scaled  = X_diab_test.copy()

# fit на train, transform на обе выборки
X_diab_train_scaled[num_cols] = diab_scaler.fit_transform(X_diab_train[num_cols])
X_diab_test_scaled[num_cols]  = diab_scaler.transform(X_diab_test[num_cols])

In [ ]:
wine_scaler = StandardScaler().set_output(transform='pandas')

X_wine_train = wine_scaler.fit_transform(X_wine_train)
X_wine_val = wine_scaler.transform(X_wine_val)

In [ ]:
# Определите правило предобработки колонок с помощью pipe_col_prep (ColumnTransformer)

pipe_col_prep = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), diab_cat_feat),
        ('num', StandardScaler(), diab_num_feat),
    ]
)

In [ ]:
# Постройте пайплайн pipe_clf
# Не забудьте зафиксировать RANDOM_STATE

pipe_clf = Pipeline(
    steps=[
        ('col_prep', pipe_col_prep),
        ('clf', LogisticRegression(random_state=RANDOM_STATE))
    ]
)

# Methods of Generation and Load data

# 1.

In [ ]:
# Сгенерируйте скрытый фактор t
# Сгенерируйте фактор x, на который влияет t
# Сгенерируйте целевую переменную f, на которую влияют факторы t и x
# Истинные коэффициенты при факторах:
#   x: 2
#   t: -4
# Переменная rng позволит зафиксировать RANDOM_STATE при генерации случайных чисел в numpy

rng = np.random.RandomState(RANDOM_STATE)

t = rng.normal(0, 1, 1000)
x = 0.5 * t + rng.normal(0, 0.5, 1000)
f = 2 * x - 4 * t + rng.normal(0, 1, 1000)

4. с помощью [make_moons](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html) 

In [ ]:
# Сгенерируйте набор данных с двумя классами и двумя признаками с помощью make_moons

features, labels = datasets.make_moons(n_samples=200, noise=0.15, random_state=RANDOM_STATE)
features = pd.DataFrame(features)
features.columns = ['x1', 'x2']

5. WITH [make_circles](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_circles.html) 

In [ ]:
from sklearn.datasets import make_circles

#make circles

X_circles, y_circles = make_circles(n_samples=1000, factor=0.1, noise=0.3, random_state=RANDOM_STATE)
X_circles = pd.DataFrame(X_circles)
X_circles.columns = ['x1', 'x2']


Function [choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html)
сгенерируйте 5 бутстрэп-подвыброк

In [ ]:
# Не забудьте зафиксировать RANDOM_STATE (с помощью переменной rng)
rng = np.random.RandomState(RANDOM_STATE)

k = 5 # Число деревьев в ансамбле (число бутстрэп-подвыброк)

train_N = len(X_titanic_train) # Размер обучающей выборки

train_idx = list(range(train_N)) # Список с индексами наблюдений обучающей выборки

# Выполните случайный выбор индексов train_idx с повторениями
titanic_bootstraps = [rng.choice(train_idx, size=train_N, replace=True) for _ in range(k)]

## 6. RNG

In [ ]:
# Сгенерируйте простой набор данных для регрессии

rng = np.random.RandomState(RANDOM_STATE)
noise = rng.normal(0, 0.18, 1000)
X = pd.DataFrame({'x': np.linspace(1, 10, 1000)})
y_true = -0.3 * X['x'] ** 0.5 * np.cos(np.pi * X['x'] / 4)
y_true = np.array(y_true)
y = y_true + noise